<div style="display: block;"><center>
<img src="panel_logo_stacked.png" width=170><br>
<h1>How to Turn Your Notebooks into Secure, Deployable Dashboards with Panel
</h1>
</center></div>

Do you have a Jupyter notebook with a plot in it?

Or a table, or an image, or an equation?

And maybe you want to explore these things, or share them?

Ok, Panel is for you.

<img src="panel_logo_stacked.png" width=170>

- Panel is new, but built on Bokeh's solid 1.x release.
- Panel lets all your notebooks double as apps or dashboards.
- Use just about any plotting library, image type, or other objects.
- Develop Python in a notebook, deploy, revise, repeat.
- Fully usable with Jupyter *and* without Jupyter installed.

## Demos

- https://panel.pyviz.org/gallery
- https://examples.pyviz.org

To dive in, let's say we have a dataset to explore, such as this<br>
[UCI ML dataset measuring the environment in a meeting room](http://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+).

In [ ]:
import pandas as pd; import numpy as np; import matplotlib.pyplot as plt

data = pd.read_csv('./occupancy.csv')
data['date'] = data.date.astype('datetime64[ns]')
data = data.set_index('date')

data.tail()

And we've written some code that smooths a time series and plots it using Matplotlib with outliers highlighted:

In [ ]:
def mpl_plot(avg, highlight):
    ax = avg.plot()
    if len(highlight): highlight.plot(style='o', ax=ax)
    fig = ax.get_figure()
    plt.close(fig)
    return fig

def find_outliers(variable='Temperature', window=30, sigma=10, view_fn=mpl_plot):
    avg = data[variable].rolling(window=window).mean()
    residual = data[variable] - avg
    std = residual.rolling(window=window).std()
    outliers = (np.abs(residual) > std * sigma)
    return view_fn(avg, avg[outliers])

We can call the function with parameters and get a plot:

In [ ]:
find_outliers(variable='Temperature', window=20, sigma=10)

It works! But exploring all these parameters by typing Python is slow and tedious. Plus we want our boss, or the boss's boss, to be able to try it out.

Let's make a panel instead:

In [1]:
import panel as pn
pn.extension()

pn.interact(find_outliers)

ModuleNotFoundError: No module named 'panel'

Let's do a bit more work and capture the full range of parameters that can be varied:

In [ ]:
kw = dict(window=(1, 60), variable=sorted(list(data.columns)), sigma=(1, 20))
pn.interact(find_outliers, **kw)

Now that we've explored it, let's share it with the boss:

In [ ]:
#pn.interact(find_outliers, **kw).show()

Boss says he doesn't know how to use the dashboard.  

Let's look at it and see how it's made, so we can add some explanatory text:

In [ ]:
i = pn.interact(find_outliers, **kw)
print(i)

Ah, it's just a column of widgets and a plot.

Let's unpack that, rearrange it, add some instructions, and hide one of the widgets that will just confuse the boss:

In [ ]:
text = "<br>\n# Room Occupancy\nSelect the variable, and the time window for smoothing"

p = pn.Row(i[1][0], pn.Column(text, i[0][0], i[0][1]))
p

Once we're happy with that in the notebook, we'll share it again:

In [ ]:
#p.show()

In [ ]:
p

Note that even widgets in another notebook cell stay linked:

In [ ]:
i[0][2]

Also note that Panel widgets are reactive, so they will update even if you set the values by hand:

In [ ]:
p

In [ ]:
i[0][0].value = 'Light'

Of course, you don't need to use the magic of `interact`; it's also easy to make widgets and link them up by hand:

In [ ]:
import panel.widgets as pnw

variable  = pnw.RadioButtonGroup(name='variable', value='Temperature', 
                                 options=list(data.columns))
window  = pnw.IntSlider(name='window', value=10, start=1, end=60)

@pn.depends(variable.param.value, window.param.value)
def reactive_outliers(variable, window):
    return find_outliers(variable, window, 10)

widgets   = pn.Column("<br>\n# Room occupancy", variable, window)
occupancy = pn.Row(reactive_outliers, widgets)

In [ ]:
occupancy

You can even define things that will display as panels without ever having your domain-specific code depend on web browsers, widgets, or other volatile technologies:

In [ ]:
import param

class RoomOccupancy(param.Parameterized):
    variable  = param.Selector(objects=list(data.columns))
    window    = param.Integer(default=10, bounds=(1, 20))
    sigma     = param.Number(default=10, bounds=(0, 20))

    def view(self):
        return find_outliers(self.variable, self.window, self.sigma)
    
obj = RoomOccupancy()
obj

And then you can make an app for whatever happens to be in `obj`, without specifying what `obj` is:

In [ ]:
pn.Row(obj.param, obj.view)

That way you can maintain a large, long-lived domain-specific codebase, supporting interactive usage alongside every other use.

So far we've only used Matplotlib.  

What about other libraries? You can use almost anything!

E.g. [hvPlot](http://hvplot.pyviz.org), a drop-in replacement for Pandas .plot() (and xarray, dask, intake, ...) that gives fully interactive [Bokeh](http://bokeh.pydata.org) plots in panels:

In [ ]:
import hvplot.pandas
def hvplot(avg, highlight):
    return avg.hvplot(height=200) * highlight.hvplot.scatter(color='orange', padding=0.1)
text2 = "## Room Occupancy\nSelect the variable and the smoothing values"
hvp   = pn.interact(find_outliers, view_fn=hvplot, **kw)
pn.Column(pn.Row(pn.panel(text2, width=400), hvp[0]), hvp[1]).servable("Occupancy")

We can bring in other datasets, dozens of other plotting libraries, linked plots, drilling down, big data, etc., but let's call that done for now. 

Can we keep it running as a server indefinitely now?  

Sure, just mark the item to serve with `.servable()` in the notebook, then run `panel serve --show Panel_Webinar0919.ipynb`. 

As you can see, Panel is designed to support your _entire_ data-analysis workflow -- work in a notebook, explore freely, deploy freely, do batch or cron jobs, and never have to rewrite to use your code in a new context.

Panel is 100% open source, but note that if you are lucky enough to be an Anaconda Enterprise customer, you can deploy Panel (or any other apps) on an isolated, authenticated container with just two clicks. 

Panel makes it dead easy to create apps from Python, and AE5 makes it dead easy to deploy them, without having to set up and configure a server, worry about security, or negotiate with IT. Just hit Deploy and go!


You can also deploy on various public servers, including AWS and free deployments on Binder and Heroku.  See panel.pyviz.org for instructions.

## Compared to Dash, Panel is:

- Typically much more concise
- More Pythonic (less reliance on HTML/CSS)
- Usable with most plotting libraries, out of the box
- Fully usable from Jupyter notebooks
- More focused on the full life cycle of exploration to deployment

## Compared to Voila+ipywidgets, Panel:

- Is not tied to Jupyter:
   * fully usable from .py, for bigger dashboards
   * deployable without Jupyter installed, for production
- Is more scalable<br>(new connections don't need overhead of a new kernel)
- Supports the full life cycle from exploration to fully standalone deployment in the same codebase
- Makes it simple to designate which output in a notebook is for the dashboard(s)

<img src="http://holoviz.org/assets/holoviz-logo-unstacked.svg" width=350>

Panel and hvPlot are HoloViz projects, like Datashader, HoloViews, GeoViews, Param, Colorcet, ...

See [holoviz.org](https://holoviz.org) to see how everything fits together to solve all your viz needs!